## XLnet rap generator

*(based on https://mccormickml.com/2019/09/19/XLNet-fine-tuning/)*

---


Setup dependencies

In [1]:
!pip install transformers
import torch
import transformers
from transformers import AutoModelWithLMHead, AutoTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split



from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

     |████████████████████████████████| 1.9MB 9.5MB/s 
     |████████████████████████████████| 3.2MB 25.1MB/s 
     |████████████████████████████████| 890kB 51.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=fcbc8f794deb2e9f467110ba052fda975c17d427e2f1611c19bfc566d82192c9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
from google.colab import files

uploaded = files.upload()

Saving lyrics.csv to lyrics.csv


Ingest data


In [4]:
df = pd.read_csv("lyrics.csv", delimiter='\t', header=None, names=['sentence_source'], encoding='latin-1')
df.shape
#df.sample(10)
#remove commas from end of lines
df2 = df.replace({',':''}, regex=True)
df2.size
#FOR TESTING
test_df = df2.sample(10000)


preprocess data for xl-net


In [5]:
sentences = test_df.sentence_source.values
sentences = [sentence + " [SEP] [CLS]" for sentence in sentences]
tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased", do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
#test_text = [tokenizer(sent, return_tensors="pt", add_special_tokens=True) for sent in sentences]
print("Tokenize the first sentence:")
print(tokenized_texts[1])
MAX_LEN = 64
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

#train_inputs = torch.tensor(test_text)
train_input_ids = torch.tensor(input_ids)
train_masks = torch.tensor(attention_masks)



Tokenize the first sentence:
['▁Mor', 'de', 'cai', '▁Rig', 'by', '▁[', 'S', 'EP', ']', '▁[', 'CL', 'S', ']']


In [6]:
batch_size = 32
#train_data = TensorDataset(tokenized_texts, train_input_ids, train_masks)
train_data = TensorDataset(train_input_ids, train_masks)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)


Train loop

In [7]:
model = AutoModelWithLMHead.from_pretrained("xlnet-base-cased", return_dict=True)
#model.cuda()


/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:970: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [8]:
from transformers import AdamW
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(optimizer_grouped_parameters,
                     lr=2e-5)

In [9]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 2

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  # Training
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    print("Step: %s"%(step))
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    # outputs = model(**inputs, labels=inputs["input_ids"])
    model.to(device)
    outputs = model(input_ids=b_input_ids, labels=b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    #print(outputs)
    loss = outputs.loss
    logits = outputs.logits
    if(loss is not None):
      train_loss_set.append(loss.item())
      perplexity = torch.exp(loss) 
      print(perplexity)   
      # Backward pass
      loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    
    # Update tracking variables
    if(loss is not None):
      tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Step: 0
tensor(3.4628, device='cuda:0', grad_fn=<ExpBackward>)
Step: 1
tensor(1.5926, device='cuda:0', grad_fn=<ExpBackward>)
Step: 2
tensor(1.6320, device='cuda:0', grad_fn=<ExpBackward>)
Step: 3
tensor(1.4172, device='cuda:0', grad_fn=<ExpBackward>)
Step: 4
tensor(1.3577, device='cuda:0', grad_fn=<ExpBackward>)
Step: 5
tensor(1.3202, device='cuda:0', grad_fn=<ExpBackward>)
Step: 6
tensor(1.2682, device='cuda:0', grad_fn=<ExpBackward>)
Step: 7
tensor(1.2474, device='cuda:0', grad_fn=<ExpBackward>)
Step: 8
tensor(1.2088, device='cuda:0', grad_fn=<ExpBackward>)
Step: 9
tensor(1.1873, device='cuda:0', grad_fn=<ExpBackward>)
Step: 10
tensor(1.1579, device='cuda:0', grad_fn=<ExpBackward>)
Step: 11
tensor(1.1620, device='cuda:0', grad_fn=<ExpBackward>)
Step: 12
tensor(1.1554, device='cuda:0', grad_fn=<ExpBackward>)
Step: 13
tensor(1.1601, device='cuda:0', grad_fn=<ExpBackward>)
Step: 14
tensor(1.1376, device='cuda:0', grad_fn=<ExpBackward>)
Step: 15
tensor(1.0993, device='cuda:0', grad_fn=<

Epoch:  50%|█████     | 1/2 [01:02<01:02, 62.11s/it]

Step: 312
tensor(1.0022, device='cuda:0', grad_fn=<ExpBackward>)
Train loss: 0.026907145739540447
Step: 0
tensor(1.0018, device='cuda:0', grad_fn=<ExpBackward>)
Step: 1
tensor(1.0021, device='cuda:0', grad_fn=<ExpBackward>)
Step: 2
tensor(1.0015, device='cuda:0', grad_fn=<ExpBackward>)
Step: 3
tensor(1.0009, device='cuda:0', grad_fn=<ExpBackward>)
Step: 4
tensor(1.0036, device='cuda:0', grad_fn=<ExpBackward>)
Step: 5
tensor(1.0017, device='cuda:0', grad_fn=<ExpBackward>)
Step: 6
tensor(1.0017, device='cuda:0', grad_fn=<ExpBackward>)
Step: 7
tensor(1.0030, device='cuda:0', grad_fn=<ExpBackward>)
Step: 8
tensor(1.0010, device='cuda:0', grad_fn=<ExpBackward>)
Step: 9
tensor(1.0039, device='cuda:0', grad_fn=<ExpBackward>)
Step: 10
tensor(1.0016, device='cuda:0', grad_fn=<ExpBackward>)
Step: 11
tensor(1.0019, device='cuda:0', grad_fn=<ExpBackward>)
Step: 12
tensor(1.0008, device='cuda:0', grad_fn=<ExpBackward>)
Step: 13
tensor(1.0015, device='cuda:0', grad_fn=<ExpBackward>)
Step: 14
tensor(

Epoch: 100%|██████████| 2/2 [01:57<00:00, 58.90s/it]

Train loss: 0.0014091198074671562


In [10]:
#DEBUG TRAIN LOOP
print(tokenized_texts)

[['▁', '"', 'You', '▁disrespect', '▁me', '▁I', "'", 'll', '▁get', '▁on', '▁board', '▁with', '▁three', '▁K', 's', '▁out', '"', '▁[', 'S', 'EP', ']', '▁[', 'CL', 'S', ']'], ['▁Mor', 'de', 'cai', '▁Rig', 'by', '▁[', 'S', 'EP', ']', '▁[', 'CL', 'S', ']'], ['▁Even', '▁in', '▁a', '▁pair', '▁of', '▁', 'cuff', 's', '▁know', '▁we', '▁always', '▁keep', '▁it', '▁real', '▁[', 'S', 'EP', ']', '▁[', 'CL', 'S', ']'], ['▁I', '▁just', '▁need', '▁to', '▁clear', '▁my', '▁mind', '▁now', '▁[', 'S', 'EP', ']', '▁[', 'CL', 'S', ']'], ['▁Me', '▁and', '▁Lon', 'nie', '▁Lynn', '▁got', '▁a', '▁lot', '▁in', '▁common', '▁[', 'S', 'EP', ']', '▁[', 'CL', 'S', ']'], ['▁', '"', 'T', 'old', '▁me', '▁a', '\x80\x9c', 'Mo', 'ney', '▁has', '▁changed', '▁you', 'a', '\x80', '?', '"', '"', 'I', '▁get', '▁to', '▁thinking', '"', '▁[', 'S', 'EP', ']', '▁[', 'CL', 'S', ']'], ['▁You', '▁lost', '▁one', '▁[', 'S', 'EP', ']', '▁[', 'CL', 'S', ']'], ['▁See', '▁the', '▁Jake', '▁don', "'", 't', '▁hit', '▁the', '▁brake', '▁just', '▁go', '

Generate using finetuned model


In [46]:
# prompt = tokenizer("<|endoftext|> All my friends are ", return_tensors="pt", add_special_tokens=True)
# generate_id = model.generate(
#     input_ids = prompt.input_ids,
#     do_sample = True,
#     max_length = 50,
#     num_beams = None,
#     top_k = 5,
#     top_p = 0.85,
#     eos_token_id = None,
#     temperature = 0.9,
#     repetition_penalty = 1.2
# )
# print(tokenizer.decode(generate_id.reshape(-1)))

PADDING_TEXT = """They tryna be crazy 
They tryna be crazy 
She wanna meet Carti 
That bitch is a Barbie 
I'ma fuck these hoes 
I'm on 730
Got a brand new pack like Kid Cudi 
I smoke dope like Kid Cudi
Push up and get the slugs from me 
I'm with all the shits 
She wanna meet Carti 
That bitch is a Barbie <eod> </s> <eos>"""

prompt = "All my friends are  "
inputs = tokenizer.encode(PADDING_TEXT + prompt, add_special_tokens=False, return_tensors="pt")
inputs = inputs.to(device)

prompt_length = len(tokenizer.decode(inputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))
outputs = model.generate(inputs, max_length=250, do_sample=True, top_p=0.99, top_k=25)
generated = prompt + tokenizer.decode(outputs[0])[prompt_length:]
print(generated)


All my friends are  iends are friends for the most part. They are friends for the most part I'm with the shits And I'm just fuck the shits In that fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fuck fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking 

In [ ]:
PADDING_TEXT = """They tryna be crazy 
They tryna be crazy 
She wanna meet Carti 
That bitch is a Barbie 
I'ma fuck these hoes 
I'm on 730
Got a brand new pack like Kid Cudi 
I smoke dope like Kid Cudi
Push up and get the slugs from me 
I'm with all the shits 
She wanna meet Carti 
That bitch is a Barbie <eod> </s> <eos>"""

for i in range(9):
  prompt = "All my friends are " + str(i) 
  inputs = tokenizer.encode(PADDING_TEXT + prompt, add_special_tokens=False, return_tensors="pt")
  inputs = inputs.to(device)
  
  prompt_length = len(tokenizer.decode(inputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))
  outputs = model.generate(inputs, max_length=250, do_sample=True, top_p=0.99, top_k=25)
  generated = prompt + tokenizer.decode(outputs[0])[prompt_length:]
  print(generated)

Generate with out the box xl-net

In [47]:
from transformers import AutoModelWithLMHead, AutoTokenizer

model = AutoModelWithLMHead.from_pretrained("xlnet-base-cased", return_dict=True)
tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")
# Padding text helps XLNet with short prompts - proposed by Aman Rusia in https://github.com/rusiaaman/XLNet-gen#methodology

PADDING_TEXT = """They tryna be cray (Mm, yeah)
They tryna be cray (Mm, yeah)
She wanna meet Carti (Carti)
That bitch is a Barbie (Yeah)
I'ma fuck these hoes (Ooh)
I'm on 730
Got a brand new pack like Kid Cudi (Brand new)
I smoke dope like Kid Cudi
Push up and get the slugs from me (Slime slime)
I'm with all the shits (Slime slime)
She wanna meet Carti (Carti)
That bitch is a Barbie (Yeah)
I'ma fuck these hoes (Ooh)
I'm on 730
Got a brand new pack like Kid Cudi (Brand new)
I smoke dope like Kid Cudi
Push up and get the<eod> </s> <eos>"""

prompt = "All my friends are "
inputs = tokenizer.encode(PADDING_TEXT + prompt, add_special_tokens=False, return_tensors="pt")
inputs = inputs.to(device)

prompt_length = len(tokenizer.decode(inputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))

model = model.to(device)
outputs = model.generate(inputs, max_length=250, do_sample=True, top_p=0.99, top_k=25)
generated = prompt + tokenizer.decode(outputs[0])[prompt_length:]

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:970: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [48]:
print(generated)

All my friends are iends are "All the men who are bad". I don't think about the things that my friends are not "bad" or who I know "I want to be" or how I need to make things happen. I don'
